## Entendendo como extrair informações sobre repositórios através da api do GitHub

Links: 
- [Search](https://developer.github.com/v3/search/)
- [Searching for repositories](https://help.github.com/en/articles/searching-for-repositories#search-by-repository-name-description-or-contents-of-the-readme-file)
- Lista de qualifiers:  https://help.github.com/en/articles/searching-code
- Documentação para search: https://developer.github.com/v3/search/

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime

Montando url de busca para os repositórios com maior quantidade de estrelas.
Usando como consulta stars:>1 para indicar que são todos os repositorios acima de 1 estrela, o que não tem impacto negativo para a busca pois existem certa de 3.305.358 repositórios e só vamos analisar 2.500.

In [2]:
q = 'q=stars:>1'
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?'
url_final = url_base+q+sort
url_final

'https://api.github.com/search/repositories?q=stars:>1&sort=stars&order=desc'

In [3]:
colunas=['id',
         'owner_type', 
           'owner_url',
           'owner_html_url',
           'html_url',
           'url',
           'fork',
           'created_at',
           'updated_at',
           'size',
           'stargazers_count',
           'language',
           'has_issues',
           'has_wiki',
           'forks_count',
           'forks',
           'open_issues',
           'watchers']

In [4]:
resultados = pd.DataFrame(columns=['id',
                                   'owner_type', 
                                   'owner_url',
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
resultados.head()

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme


In [5]:
def add_resultado(item):
    df = pd.DataFrame([[
                        item.get('id'),
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None)
                    ]], columns=colunas)

    return df    

In [6]:
def extrair_dados(data, resultados):
    for item in data.get('items', None):
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)
        
    return resultados

In [7]:
def get_total_paginas(data):
    
    itens_por_pagina = len(data.get('items'))
    total_paginas = data.get('total_count') // itens_por_pagina # opreração de div em python
    total_paginas

    print('Total de registros:{0} , Registros por página:{1}, Total de Páginas:{2}'.format(
        data.get('total_count'), itens_por_pagina, total_paginas))
    
    return total_paginas        

In [8]:
def percorrendo_paginas(url):
    
    resultados = pd.DataFrame(columns=['id', 
                                       'owner_type', 
                                   'owner_url',
                                   'owner_html_url',
                                   'html_url',
                                   'url',
                                   'fork',
                                   'created_at',
                                   'updated_at',
                                   'size',
                                   'stargazers_count',
                                   'language',
                                   'has_issues',
                                   'has_wiki',
                                   'forks_count',
                                   'forks',
                                   'open_issues',
                                   'watchers',
                                   'commits',
                                   'contributors',
                                   'readme',])
    
    print('Extraindo página:1')
    results = requests.get(url)    
    data = dict(results.json())
    resultados = extrair_dados(data, resultados)

    # iteracoes = get_total_paginas(data)
    iteracoes = 84 # Para pegar em torno de 2500 registros, semelhante ao artigo original
    
    for iteracao in range(1, 34):
        print("\n>>>>>>>> Iteracao:{0}".format(iteracao+1))
        print("Tempo atual:{0}".format(datetime.now()))

        # Para requisições não autenticadas a api restringe para 10 requisições por minuto,
        # Para requisições autenticadas 30 por minuto.
        if iteracao in [10, 20, 30, 40, 50, 60, 70, 80]:
            print("sleep 1 minuto")
            time.sleep(60)
        
        header = dict(results.links)
        next_url = header.get('next').get('url')
        print("Next url extraída: {0}".format(next_url))
        
        print('Extraindo página:{0}'.format(iteracao+1))
        results = requests.get(next_url)
        print('Status:{0}'.format(results))
        
        data = dict(results.json())
            
        resultados = extrair_dados(data, resultados)
        
    return resultados

In [ ]:
%%time
resultados = percorrendo_paginas(url_final)

In [20]:
resultados.describe()

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme
count,1020,1020,1020,1020,1020,1020,1020,1020,1020,1020,...,907,1020,1020,1020,1020,1020,1020,0.0,0.0,0.0
unique,812,2,691,691,812,812,1,812,800,801,...,36,2,2,752,752,399,780,0.0,0.0,0.0
top,69662720,Organization,https://api.github.com/users/google,https://github.com/google,https://github.com/froala/design-blocks,https://api.github.com/repos/mxgmn/WaveFunctio...,False,2010-09-22T06:16:55Z,2019-07-03T19:58:43Z,358204,...,JavaScript,True,True,1016,1016,2,21086,NaN,NaN,NaN
freq,4,625,21,21,4,4,1020,4,5,4,...,323,966,678,7,7,21,7,NaN,NaN,NaN


In [23]:
resultados.to_csv('../dados/repositorios.csv', index=False)

"Only the first 1000 search results are available"

In [26]:
page_35 = requests.get("https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35")
page_35.json()

{'documentation_url': 'https://developer.github.com/v3/search/',
 'message': 'Only the first 1000 search results are available'}

In [27]:
resultados.columns.values

array(['id', 'owner_type', 'owner_url', 'owner_html_url', 'html_url',
       'url', 'fork', 'created_at', 'updated_at', 'size',
       'stargazers_count', 'language', 'has_issues', 'has_wiki',
       'forks_count', 'forks', 'open_issues', 'watchers', 'commits',
       'contributors', 'readme'], dtype=object)

In [ ]:
resultados['stargazers_count'].sort_values()

Percorrendo nova consulta por repositorios com estrelas acima abaixo do menor número coletado anteriormente - 10662

In [21]:
q = 'q=stars:<10663'
nova_pesquisa = url_base+q+sort
nova_pesquisa

'https://api.github.com/search/repositories?q=stars:<10663&sort=stars&order=desc'

In [22]:
%%time
resultados = percorrendo_paginas(nova_pesquisa)

Extraindo página:1

>>>>>>>> Iteracao:2
Tempo atual:2019-07-05 00:08:54.308947
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3C10663&sort=stars&order=desc&page=2
Extraindo página:2
Status:<Response [200]>

>>>>>>>> Iteracao:3
Tempo atual:2019-07-05 00:08:57.272899
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3C10663&sort=stars&order=desc&page=3
Extraindo página:3
Status:<Response [200]>

>>>>>>>> Iteracao:4
Tempo atual:2019-07-05 00:09:00.560360
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3C10663&sort=stars&order=desc&page=4
Extraindo página:4
Status:<Response [200]>

>>>>>>>> Iteracao:5
Tempo atual:2019-07-05 00:09:04.461887
Next url extraída: https://api.github.com/search/repositories?q=stars%3A%3C10663&sort=stars&order=desc&page=5
Extraindo página:5
Status:<Response [200]>

>>>>>>>> Iteracao:6
Tempo atual:2019-07-05 00:09:06.373829
Next url extraída: https://api.github.com/search/repositories?q=stars

In [23]:
resultados.describe()

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme
count,1020,1020,1020,1020,1020,1020,1020,1020,1020,1020,...,953,1020,1020,1020,1020,1020,1020,0.0,0.0,0.0
unique,723,2,709,709,723,723,1,723,723,571,...,35,2,2,80,80,46,93,0.0,0.0,0.0
top,6089223,User,https://api.github.com/users/drewconway,https://github.com/maccman,https://github.com/allenhsu/PortalDemo,https://api.github.com/repos/raix/Meteor-jshintrc,False,2012-11-27T21:16:26Z,2017-10-04T05:42:27Z,160,...,JavaScript,True,True,6,6,0,74,NaN,NaN,NaN
freq,3,794,6,6,3,3,1020,3,3,9,...,281,985,916,54,54,270,89,NaN,NaN,NaN


In [24]:
resultados.to_csv('../dados/repositorios_nova_pesquisa.csv', index=False)

## Extraindo commits e contributors através da biblioteca PyGithub

Links:
- https://github.com/PyGithub/PyGithub

In [9]:
from github import Github
from github import Repository
from github import ContentFile
from bs4 import BeautifulSoup as bs

In [10]:
token = 'token'

In [11]:
g = Github(token)

In [25]:
resultados_csv = pd.read_csv('../dados/repositorios_update.csv')
resultados_csv.head()

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme
0,28457823,Organization,https://api.github.com/users/freeCodeCamp,https://github.com/freeCodeCamp,https://github.com/freeCodeCamp/freeCodeCamp,https://api.github.com/repos/freeCodeCamp/free...,False,2014-12-24T17:49:19Z,2019-07-01T20:27:35Z,117391,...,JavaScript,True,False,22169,22169,1489,303613,23674.0,3672,NaN
1,177736533,User,https://api.github.com/users/996icu,https://github.com/996icu,https://github.com/996icu/996.ICU,https://api.github.com/repos/996icu/996.ICU,False,2019-03-26T07:31:14Z,2019-07-01T16:02:38Z,59169,...,Rust,False,False,21334,21334,16695,246365,2904.0,579,NaN
2,11730342,Organization,https://api.github.com/users/vuejs,https://github.com/vuejs,https://github.com/vuejs/vue,https://api.github.com/repos/vuejs/vue,False,2013-07-29T03:24:51Z,2019-07-01T18:52:14Z,27673,...,JavaScript,True,True,20575,20575,307,142614,3024.0,276,NaN
3,2126244,Organization,https://api.github.com/users/twbs,https://github.com/twbs,https://github.com/twbs/bootstrap,https://api.github.com/repos/twbs/bootstrap,False,2011-07-29T21:19:00Z,2019-07-01T19:09:47Z,143722,...,JavaScript,True,False,65902,65902,365,134321,18744.0,1078,NaN
4,10270250,Organization,https://api.github.com/users/facebook,https://github.com/facebook,https://github.com/facebook/react,https://api.github.com/repos/facebook/react,False,2013-05-24T16:15:54Z,2019-07-01T19:57:38Z,143031,...,JavaScript,True,True,24380,24380,740,131890,11104.0,1301,NaN


In [26]:
resultados_csv[355:]

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme
355,90528830,User,https://api.github.com/users/Solido,https://github.com/Solido,https://github.com/Solido/awesome-flutter,https://api.github.com/repos/Solido/awesome-fl...,False,2017-05-07T11:45:27Z,2019-07-01T19:40:07Z,1372,...,Dart,False,False,2428,2428,27,19302,1189.0,173,NaN
356,18840003,Organization,https://api.github.com/users/kriasoft,https://github.com/kriasoft,https://github.com/kriasoft/react-starter-kit,https://api.github.com/repos/kriasoft/react-st...,False,2014-04-16T13:08:18Z,2019-07-01T14:13:06Z,4622,...,JavaScript,True,True,3879,3879,486,19294,NaN,NaN,NaN
357,4037197,Organization,https://api.github.com/users/ycm-core,https://github.com/ycm-core,https://github.com/ycm-core/YouCompleteMe,https://api.github.com/repos/ycm-core/YouCompl...,False,2012-04-16T03:12:14Z,2019-07-01T18:53:40Z,34969,...,Python,True,True,2153,2153,52,19293,NaN,NaN,NaN
358,19141383,User,https://api.github.com/users/fouber,https://github.com/fouber,https://github.com/fouber/blog,https://api.github.com/repos/fouber/blog,False,2014-04-25T09:44:42Z,2019-07-01T10:27:13Z,19107,...,NaN,True,True,2186,2186,11,19263,NaN,NaN,NaN
359,60325062,User,https://api.github.com/users/terryum,https://github.com/terryum,https://github.com/terryum/awesome-deep-learni...,https://api.github.com/repos/terryum/awesome-d...,False,2016-06-03T06:48:30Z,2019-07-01T17:50:13Z,147,...,TeX,True,True,3703,3703,28,19196,NaN,NaN,NaN
360,8859474,User,https://api.github.com/users/skylot,https://github.com/skylot,https://github.com/skylot/jadx,https://api.github.com/repos/skylot/jadx,False,2013-03-18T17:08:21Z,2019-07-01T14:11:30Z,13473,...,Java,True,True,2108,2108,90,19137,NaN,NaN,NaN
361,6007295,User,https://api.github.com/users/fxsjy,https://github.com/fxsjy,https://github.com/fxsjy/jieba,https://api.github.com/repos/fxsjy/jieba,False,2012-09-29T07:52:01Z,2019-07-01T16:22:31Z,44251,...,Python,True,True,4969,4969,482,19127,NaN,NaN,NaN
362,12244426,User,https://api.github.com/users/rstacruz,https://github.com/rstacruz,https://github.com/rstacruz/nprogress,https://api.github.com/repos/rstacruz/nprogress,False,2013-08-20T13:58:02Z,2019-07-01T14:19:58Z,344,...,JavaScript,True,True,1588,1588,114,19122,NaN,NaN,NaN
363,790359,Organization,https://api.github.com/users/sequelize,https://github.com/sequelize,https://github.com/sequelize/sequelize,https://api.github.com/repos/sequelize/sequelize,False,2010-07-22T07:11:11Z,2019-07-01T17:54:03Z,25711,...,JavaScript,True,False,3005,3005,537,19120,NaN,NaN,NaN
364,33895378,User,https://api.github.com/users/bevacqua,https://github.com/bevacqua,https://github.com/bevacqua/dragula,https://api.github.com/repos/bevacqua/dragula,False,2015-04-13T21:35:38Z,2019-07-01T12:56:36Z,1832,...,JavaScript,True,False,1639,1639,212,19092,NaN,NaN,NaN


In [27]:
id_repos = resultados_csv.id
len(id_repos[355:])

665

In [23]:
def get_contributors(url_repo):
    page = requests.get(url_repo)
    page_parse = bs(page.text, 'html.parser')
    lista = page_parse.findAll('span', attrs={'class': 'num text-emphasized'}) 
    
    text = lista[3].text
    contributors = text.replace(" ", "").replace("\n", "").replace(",", "")
    #contributors = int(contributors)
    
    return(contributors)

In [28]:
it = 355
for id_repo in id_repos[356:]:
    
    it = it + 1
    print("\nIteração: ", it)
    
    if it % 10 == 0:
        print("sleep 1 minuto")
        time.sleep(60)

    repo = g.get_repo(id_repo)
    print("Nome do repositório:{0}".format(repo.name))

    commits = repo.get_commits().totalCount
    print("Commits: ", commits)

    url_repo = resultados_csv.loc[resultados_csv["id"] == id_repo]['html_url'].to_list()[0]
    print("Url: ", url_repo)

    contributors = get_contributors(url_repo)
    print("Contributors: ", contributors)

    print("Atualizando data frame :)")
    resultados_csv.loc[resultados_csv["id"] == id_repo, 'commits'] = commits 
    resultados_csv.loc[resultados_csv["id"] == id_repo, 'contributors'] = contributors

    resultados_csv.to_csv('../dados/repositorios_update.csv', index=False)


Iteração:  356
Nome do repositório:react-starter-kit
Commits:  915
Url:  https://github.com/kriasoft/react-starter-kit
Contributors:  
Atualizando data frame :)

Iteração:  357
Nome do repositório:YouCompleteMe
Commits:  2532
Url:  https://github.com/ycm-core/YouCompleteMe
Contributors:  135
Atualizando data frame :)

Iteração:  358
Nome do repositório:blog
Commits:  94
Url:  https://github.com/fouber/blog
Contributors:  
Atualizando data frame :)

Iteração:  359
Nome do repositório:awesome-deep-learning-papers
Commits:  168
Url:  https://github.com/terryum/awesome-deep-learning-papers
Contributors:  30
Atualizando data frame :)

Iteração:  360
sleep 1 minuto
Nome do repositório:jadx
Commits:  1049
Url:  https://github.com/skylot/jadx
Contributors:  33
Atualizando data frame :)

Iteração:  361
Nome do repositório:jieba
Commits:  500
Url:  https://github.com/fxsjy/jieba
Contributors:  
Atualizando data frame :)

Iteração:  362
Nome do repositório:nprogress
Commits:  205
Url:  https://g

Commits:  243
Url:  https://github.com/SamyPesse/How-to-Make-a-Computer-Operating-System
Contributors:  
Atualizando data frame :)

Iteração:  409
Nome do repositório:intro.js
Commits:  631
Url:  https://github.com/usablica/intro.js
Contributors:  74
Atualizando data frame :)

Iteração:  410
sleep 1 minuto
Nome do repositório:awesome-javascript
Commits:  573
Url:  https://github.com/sorrycc/awesome-javascript
Contributors:  169
Atualizando data frame :)

Iteração:  411
Nome do repositório:mocha
Commits:  3190
Url:  https://github.com/mochajs/mocha
Contributors:  415
Atualizando data frame :)

Iteração:  412
Nome do repositório:spacemacs
Commits:  7431
Url:  https://github.com/syl20bnr/spacemacs
Contributors:  557
Atualizando data frame :)

Iteração:  413
Nome do repositório:mastodon
Commits:  7458
Url:  https://github.com/tootsuite/mastodon
Contributors:  567
Atualizando data frame :)

Iteração:  414
Nome do repositório:js-stack-from-scratch
Commits:  566
Url:  https://github.com/verek

Nome do repositório:vim
Commits:  10152
Url:  https://github.com/vim/vim
Contributors:  1
Atualizando data frame :)

Iteração:  462
Nome do repositório:hosts
Commits:  1260
Url:  https://github.com/kelthuzadx/hosts
Contributors:  77
Atualizando data frame :)

Iteração:  463
Nome do repositório:normalizr
Commits:  416
Url:  https://github.com/paularmstrong/normalizr
Contributors:  52
Atualizando data frame :)

Iteração:  464
Nome do repositório:lottie-web
Commits:  1879
Url:  https://github.com/airbnb/lottie-web
Contributors:  47
Atualizando data frame :)

Iteração:  465
Nome do repositório:react-select
Commits:  3285
Url:  https://github.com/JedWatson/react-select
Contributors:  335
Atualizando data frame :)

Iteração:  466
Nome do repositório:Sortable
Commits:  984
Url:  https://github.com/SortableJS/Sortable
Contributors:  118
Atualizando data frame :)

Iteração:  467
Nome do repositório:alacritty
Commits:  1258
Url:  https://github.com/jwilm/alacritty
Contributors:  243
Atualizando 

Contributors:  
Atualizando data frame :)

Iteração:  516
Nome do repositório:fonts
Commits:  202
Url:  https://github.com/powerline/fonts
Contributors:  59
Atualizando data frame :)

Iteração:  517
Nome do repositório:windows95
Commits:  111
Url:  https://github.com/felixrieseberg/windows95
Contributors:  11
Atualizando data frame :)

Iteração:  518
Nome do repositório:ghidra
Commits:  623
Url:  https://github.com/NationalSecurityAgency/ghidra
Contributors:  52
Atualizando data frame :)

Iteração:  519
Nome do repositório:mostly-adequate-guide
Commits:  793
Url:  https://github.com/MostlyAdequate/mostly-adequate-guide
Contributors:  
Atualizando data frame :)

Iteração:  520
sleep 1 minuto
Nome do repositório:flux
Commits:  392
Url:  https://github.com/facebook/flux
Contributors:  113
Atualizando data frame :)

Iteração:  521
Nome do repositório:SpinKit
Commits:  213
Url:  https://github.com/tobiasahlin/SpinKit
Contributors:  15
Atualizando data frame :)

Iteração:  522
Nome do reposi

Contributors:  269
Atualizando data frame :)

Iteração:  570
sleep 1 minuto
Nome do repositório:MBProgressHUD
Commits:  621
Url:  https://github.com/jdg/MBProgressHUD
Contributors:  
Atualizando data frame :)

Iteração:  571
Nome do repositório:algorithms
Commits:  799
Url:  https://github.com/keon/algorithms
Contributors:  
Atualizando data frame :)

Iteração:  572
Nome do repositório:tutorials
Commits:  13768
Url:  https://github.com/eugenp/tutorials
Contributors:  518
Atualizando data frame :)

Iteração:  573
Nome do repositório:coffeescript
Commits:  4732
Url:  https://github.com/jashkenas/coffeescript
Contributors:  
Atualizando data frame :)

Iteração:  574
Nome do repositório:bower
Commits:  2715
Url:  https://github.com/bower/bower
Contributors:  
Atualizando data frame :)

Iteração:  575
Nome do repositório:awesome-vscode
Commits:  535
Url:  https://github.com/viatsko/awesome-vscode
Contributors:  138
Atualizando data frame :)

Iteração:  576
Nome do repositório:mattermost-ser

...

Contributors:  61
Atualizando data frame :)

Iteração:  787
Nome do repositório:airflow
Commits:  6549
Url:  https://github.com/apache/airflow
Contributors:  845
Atualizando data frame :)

Iteração:  788
Nome do repositório:Material-Animations
Commits:  76
Url:  https://github.com/lgvalle/Material-Animations
Contributors:  
Atualizando data frame :)

Iteração:  789
Nome do repositório:git-extras
Commits:  1315
Url:  https://github.com/tj/git-extras
Contributors:  
Atualizando data frame :)

Iteração:  790
sleep 1 minuto
Nome do repositório:celery
Commits:  10957
Url:  https://github.com/celery/celery
Contributors:  735
Atualizando data frame :)

Iteração:  791
Nome do repositório:v8
Commits:  57033
Url:  https://github.com/v8/v8
Contributors:  327
Atualizando data frame :)

Iteração:  792
Nome do repositório:amphtml
Commits:  13393
Url:  https://github.com/ampproject/amphtml
Contributors:  731
Atualizando data frame :)

Iteração:  793
Nome do repositório:MJRefresh
Commits:  319
Url:  h

Nome do repositório:fonts
Commits:  1481
Url:  https://github.com/google/fonts
Contributors:  30
Atualizando data frame :)

Iteração:  841
Nome do repositório:dep
Commits:  3322
Url:  https://github.com/golang/dep
Contributors:  178
Atualizando data frame :)

Iteração:  842
Nome do repositório:forever
Commits:  647
Url:  https://github.com/foreversd/forever
Contributors:  79
Atualizando data frame :)

Iteração:  843
Nome do repositório:flatbuffers
Commits:  1697
Url:  https://github.com/google/flatbuffers
Contributors:  
Atualizando data frame :)

Iteração:  844
Nome do repositório:cheatsheets-ai
Commits:  26
Url:  https://github.com/kailashahirwar/cheatsheets-ai
Contributors:  
Atualizando data frame :)

Iteração:  845
Nome do repositório:kibana
Commits:  26122
Url:  https://github.com/elastic/kibana
Contributors:  390
Atualizando data frame :)

Iteração:  846
Nome do repositório:leetcode
Commits:  228
Url:  https://github.com/azl397985856/leetcode
Contributors:  14
Atualizando data f

Nome do repositório:selectize.js
Commits:  692
Url:  https://github.com/selectize/selectize.js
Contributors:  78
Atualizando data frame :)

Iteração:  895
Nome do repositório:rancher
Commits:  2857
Url:  https://github.com/rancher/rancher
Contributors:  60
Atualizando data frame :)

Iteração:  896
Nome do repositório:vegeta
Commits:  628
Url:  https://github.com/tsenart/vegeta
Contributors:  43
Atualizando data frame :)

Iteração:  897
Nome do repositório:nerdtree
Commits:  1175
Url:  https://github.com/scrooloose/nerdtree
Contributors:  100
Atualizando data frame :)

Iteração:  898
Nome do repositório:vivus
Commits:  169
Url:  https://github.com/maxwellito/vivus
Contributors:  
Atualizando data frame :)

Iteração:  899
Nome do repositório:symphony
Commits:  6643
Url:  https://github.com/b3log/symphony
Contributors:  
Atualizando data frame :)

Iteração:  900
sleep 1 minuto
Nome do repositório:predictionio
Commits:  4482
Url:  https://github.com/apache/predictionio
Contributors:  
Atua

Contributors:  
Atualizando data frame :)

Iteração:  948
Nome do repositório:fabric
Commits:  1060
Url:  https://github.com/fabric/fabric
Contributors:  7
Atualizando data frame :)

Iteração:  949
Nome do repositório:matomo
Commits:  25823
Url:  https://github.com/matomo-org/matomo
Contributors:  225
Atualizando data frame :)

Iteração:  950
sleep 1 minuto
Nome do repositório:react-native-vector-icons
Commits:  486
Url:  https://github.com/oblador/react-native-vector-icons
Contributors:  88
Atualizando data frame :)

Iteração:  951
Nome do repositório:bootswatch
Commits:  1701
Url:  https://github.com/thomaspark/bootswatch
Contributors:  
Atualizando data frame :)

Iteração:  952
Nome do repositório:CocoaLumberjack
Commits:  1740
Url:  https://github.com/CocoaLumberjack/CocoaLumberjack
Contributors:  155
Atualizando data frame :)

Iteração:  953
Nome do repositório:redox
Commits:  2104
Url:  https://github.com/redox-os/redox
Contributors:  
Atualizando data frame :)

Iteração:  954
No

Commits:  35706
Url:  https://github.com/Automattic/wp-calypso
Contributors:  436
Atualizando data frame :)

Iteração:  1002
Nome do repositório:micro
Commits:  1771
Url:  https://github.com/zyedidia/micro
Contributors:  107
Atualizando data frame :)

Iteração:  1003
Nome do repositório:jstips
Commits:  412
Url:  https://github.com/loverajoel/jstips
Contributors:  
Atualizando data frame :)

Iteração:  1004
Nome do repositório:numpy
Commits:  20740
Url:  https://github.com/numpy/numpy
Contributors:  785
Atualizando data frame :)

Iteração:  1005
Nome do repositório:vim-fugitive
Commits:  1033
Url:  https://github.com/tpope/vim-fugitive
Contributors:  77
Atualizando data frame :)

Iteração:  1006
Nome do repositório:cli
Commits:  905
Url:  https://github.com/urfave/cli
Contributors:  
Atualizando data frame :)

Iteração:  1007
Nome do repositório:smartcrop.js
Commits:  116
Url:  https://github.com/jwagner/smartcrop.js
Contributors:  
Atualizando data frame :)

Iteração:  1008
Nome do re

In [30]:
resultados_csv.tail()

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,commits,contributors,readme
1015,45896779,Organization,https://api.github.com/users/jikexueyuanwiki,https://github.com/jikexueyuanwiki,https://github.com/jikexueyuanwiki/tensorflow-zh,https://api.github.com/repos/jikexueyuanwiki/t...,False,2015-11-10T07:59:30Z,2019-07-01T13:37:41Z,44266,...,TeX,True,True,4091,4091,28,10964,714.0,,NaN
1016,58327877,Organization,https://api.github.com/users/browsh-org,https://github.com/browsh-org,https://github.com/browsh-org/browsh,https://api.github.com/repos/browsh-org/browsh,False,2016-05-08T19:34:15Z,2019-07-01T16:17:08Z,7371,...,JavaScript,True,True,306,306,108,10957,372.0,18,NaN
1017,63935685,User,https://api.github.com/users/gztchan,https://github.com/gztchan,https://github.com/gztchan/awesome-design,https://api.github.com/repos/gztchan/awesome-d...,False,2016-07-22T08:06:02Z,2019-07-01T19:10:42Z,241,...,NaN,True,True,877,877,8,10957,194.0,,NaN
1018,4702560,Organization,https://api.github.com/users/PHPOffice,https://github.com/PHPOffice,https://github.com/PHPOffice/PHPExcel,https://api.github.com/repos/PHPOffice/PHPExcel,False,2012-06-18T15:30:27Z,2019-07-01T08:55:02Z,22024,...,PHP,True,True,3805,3805,668,10956,1274.0,61,NaN
1019,14734876,Organization,https://api.github.com/users/eclipse,https://github.com/eclipse,https://github.com/eclipse/deeplearning4j,https://api.github.com/repos/eclipse/deeplearn...,False,2013-11-27T02:03:28Z,2019-07-01T16:14:43Z,617235,...,Java,True,True,4691,4691,844,10941,10.0,5,NaN


In [31]:
resultados_csv.to_csv('../dados/repositorios_update.csv', index=False)

Pegando readme

In [ ]:
repo = g.get_repo("freeCodeCamp")
contents = repo.get_contents("README.md")
print(contents.decoded_content)